In [1]:
import os 

In [2]:
%pwd

'd:\\mlops\\end-to-end-mlops-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlops\\end-to-end-mlops-project'

update the entity 

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
  root_dir : Path
  train_data_path : Path
  test_data_path : Path
  model_name : str
  alpha : float 
  l1_ratio : float 
  target_column :str 

update the configuration 

In [6]:
from mlProject import logger
from mlProject.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath =Path('D:\mlops\end-to-end-mlops-project\config\config.yaml'),
        params_filepath = Path("D:\mlops\end-to-end-mlops-project\params.yaml"),
        schema_filepath = Path("D:\mlops\end-to-end-mlops-project\schema.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self )-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path , 
            model_name=config.model_name,
            alpha=params.alpha ,
            l1_ratio=params.l1_ratio,
            target_column=schema.name

        )
        return model_trainer_config

In [14]:
import os 
import pandas as pd
from mlProject import logger
import joblib
from sklearn.linear_model import ElasticNet

In [15]:
class ModelTrainer:
    def __init__(self, config : ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column],axis=1)
        X_test = test_data.drop([self.config.target_column],axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha , l1_ratio= self.config.l1_ratio,random_state=42)
        lr.fit(X_train,y_train)
        joblib.dump(lr,os.path.join(self.config.root_dir , self.config.model_name))

In [16]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-03-29 14:43:15,008: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\config\config.yaml loaded successfully]
[2024-03-29 14:43:15,010: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\params.yaml loaded successfully]
[2024-03-29 14:43:15,013: INFO : common : yaml file: D:\mlops\end-to-end-mlops-project\schema.yaml loaded successfully]
[2024-03-29 14:43:15,015: INFO : common : created directory at: artifacts]
[2024-03-29 14:43:15,016: INFO : common : created directory at: artifacts/model_trainer]
